Setting up the environment

Import modules and initialize

In [1]:
import os
from dotenv import load_dotenv

# 1. Load the .env file BEFORE doing anything else
load_dotenv() 

# 2. Check if the key is actually there (for debugging)
if not os.getenv("OPENAI_API_KEY"):
    print("❌ ERROR: OPENAI_API_KEY not found in .env!")
else:
    print("✅ API Key loaded successfully.")

# 3. NOW import your custom modules
from src.data_processor import process_input
from src.llm_processor import generate_script
from src.tts_generator import generate_audio

✅ API Key loaded successfully.


Setting Up Open AI API key access

In [2]:
import os
from dotenv import load_dotenv

# The "../" tells Python to look one folder up (in the WEEK 1 root)
load_dotenv("../.env")

# Retrieve the key
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    print("✅ API Key successfully loaded!")
else:
    print("❌ API Key not found. Check your .env file location.")

✅ API Key successfully loaded!


Step 3: Text Pre-processing & Chunking

Ensure the text fits within API character limits.

In [3]:
import re
import os

# Load text from Text input folder
rec_dir = 'Text input'
txts = [f for f in os.listdir(rec_dir) if f.lower().endswith('.txt')]
if not txts:
    raise FileNotFoundError(f"No .txt files found in {rec_dir!r}")

fname = os.path.join(rec_dir, txts[0])
with open(fname, 'r', encoding='utf-8') as f:
    full_text = f.read()

print(f"✅ Loaded {len(full_text)} characters from {fname}")

def chunk_text_by_sentences(text, max_chars=4000):
    """
    Splits text into chunks of max_chars, ensuring we don't 
    break sentences in the middle.
    """
    # 1. Basic cleaning: remove extra whitespace/newlines
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 2. Split text into sentences using regex
    sentences = re.split(r'(?<=[.!?]) +', text)
    
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) + 1 <= max_chars:
            current_chunk += (sentence + " ")
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "
    
    if current_chunk:
        chunks.append(current_chunk.strip())
        
    return chunks

text_chunks = chunk_text_by_sentences(full_text)

print(f"Processed {len(text_chunks)} chunks for TTS.")
for i, chunk in enumerate(text_chunks):
    print(f"Chunk {i+1} length: {len(chunk)} characters")

✅ Loaded 5386 characters from Text input\supplements_scorecard_notes.txt
Processed 2 chunks for TTS.
Chunk 1 length: 3949 characters
Chunk 2 length: 1417 characters


Define the logic wrapper

In [4]:
def create_podcast(input_text, speaker_voice):
    # 1. Process and generate script
    cleaned_data = process_input(input_text)
    script = generate_script(cleaned_data)
    
    # 2. Pass both script AND voice to the generator
    audio_path = generate_audio(script, voice=speaker_voice) 
    
    return script, audio_path

Build the ui with blocks

In [5]:
import gradio as gr
import os

# Load the text file from Text input folder
text_file_path = os.path.join('Text input', 'supplements_scorecard_notes.txt')
with open(text_file_path, 'r', encoding='utf-8') as f:
    loaded_text = f.read()

with gr.Blocks(title="AI Podcast Studio") as demo:
    gr.Markdown("# 🎙️ AI Podcast Studio")
    gr.Markdown("Transform your notes or URLs into a fully produced podcast.")
    
    with gr.Row():
        with gr.Column():
            # Inputs - pre-populate with loaded text
            input_data = gr.Textbox(label="Source Content", placeholder="Paste your article or notes here...", lines=10, value=loaded_text)
            voice_opt = gr.Dropdown(choices=["Alloy", "Echo", "Shimmer"], label="Select Voice", value="Alloy")
            generate_btn = gr.Button("Generate Podcast", variant="primary")
            
            gr.Examples(
            examples=[["The history of the internet", "Echo"], ["Benefits of Vitamin D", "Shimmer"]],
            inputs=[input_data, voice_opt]
            )

        with gr.Column():
            # Outputs
            script_out = gr.Textbox(label="Generated Script", interactive=False, lines=10)
            audio_out = gr.Audio(label="Produced Podcast")

    # Link the button to the function
    generate_btn.click(
        fn=create_podcast,
        inputs=[input_data, voice_opt],
        outputs=[script_out, audio_out]
    )

# Launch with queue and no share
demo.queue().launch(share=False, show_error=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


3. How the Data Flows

Input: The user pastes text into the input_data box.

Trigger: The generate_btn click sends that text to create_podcast.

Processing: * llm_processor.py turns the text into a dialogue.

tts_generator.py saves an .mp3 file to a temporary directory.

Output: Gradio updates the script_out text box and the audio_out player simultaneously.

Adding templates